<a href="https://colab.research.google.com/github/VictorFu0717/AI/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import  ZipFile
f = ZipFile("/content/drive/MyDrive/chinese_news_trans.zip")
f.extractall()
f = ZipFile("/content/drive/MyDrive/chinese_news_test.zip")
f.extractall()
#解壓縮檔案

In [ ]:
import os
import glob
import pandas as pd
def process(base):
    datas = {
        "content":[],
        "ans":[]
    }
    dirs = glob.glob(os.path.join(base, "*"))
    for d in dirs:
        fns1 = os.path.join(d, "*.txt")
        fns2 = os.path.join(d, "*.TXT")
        fns = glob.glob(fns1) + glob.glob(fns2)
        for fn in fns:
            with open(fn, "r", encoding="utf-8") as f:
                datas["content"].append(f.read())
            datas["ans"].append(os.path.split(d)[-1])
    df = pd.DataFrame(datas)
    return df
train_df = process("chinese_news_trans")
test_df = process("chinese_news_test")

In [ ]:
test_df

,content,ans
0,空襲巴格達突出了B-1的新用途並證明該機效率 \n\n-------------------...,軍事
1,軍事評論：“巴格達之戰”考驗美軍巷戰武器裝備(1) \n\n----------------...,軍事
2,軍事評論：美英聯軍地面作戰裝備特點分析(2) \n\n-------------------...,軍事
3,在伊拉克戰爭中使用的空襲精確制導彈藥(1) \n\n--------------------...,軍事
4,美海軍成功演示飛機在空中控制無人機(附圖) \n\n--------------------...,軍事
...,...,...
96,金日成再次當選為國家主席\n新華社平壤５月２４日電（記者黎能清）朝鮮民...,政治
97,楊主席提出促進中拉關係的５項建議\n新華社蒙得維的亞５月２３日電正在這...,政治
98,阿拉伯外長會議閉幕\n新華社巴格達５月２４日電為期２天的阿拉伯外長會\...,政治
99,ＡＡ０５５\n金日成指出朝鮮北方堅持以聯邦制統一國家\n (4)內容: ...,政治


In [ ]:
u = train_df["ans"].unique()
name2cat = {name:i for i, name in enumerate(u)}
cat2name = {i:name for i, name in enumerate(u)}
# 準備反向字典
y_train = train_df["ans"].replace(name2cat)
y_test = test_df["ans"].replace(name2cat)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: ans, Length: 101, dtype: int64

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")

('dict.txt.big', <http.client.HTTPMessage at 0x7fa5fd565ed0>)

In [ ]:
import jieba
jieba.set_dictionary("dict.txt.big")
#載入大字典來分詞
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test


Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.618 seconds.
Prefix dict has been built successfully.


0      空襲 巴格達 突出 了 B - 1 的 新 用途 並 證明 該機 效率   \n \n --...
1      軍事 評論 ： “ 巴格達 之戰 ” 考驗 美軍 巷戰 武器裝備 ( 1 )   \n \n...
2      軍事 評論 ： 美英 聯軍 地面 作戰 裝備 特點 分析 ( 2 )   \n \n ---...
3      在 伊拉克 戰爭 中 使用 的 空襲 精確 制導 彈藥 ( 1 )   \n \n ----...
4      美海軍 成功 演示 飛機 在 空中 控制 無人機 ( 附圖 )   \n \n ------...
                             ...                        
96                           金日成 再次 當選 為 國家 主席 \n 新華社...
97                           楊 主席 提出 促進 中 拉關係 的 ５ 項 建...
98                           阿拉伯 外長 會議 閉幕 \n 新華社 巴格達 ...
99                   Ａ Ａ ０ ５ ５ \n 金日成 指出 朝鮮 北方 堅持 以 聯...
100                          外交部 發言人 談 克什米爾 事態 發展 \n ...
Name: content, Length: 101, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
vec.vocabulary_
x_train_count

<2637x76399 sparse matrix of type '<class 'numpy.int64'>'
	with 448436 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#alpha給資料保底的數值
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
# loss正確率和預測的誤差
pre = clf.predict(x_test_count)
print(accuracy_score(y_test, pre))
prob = clf.predict_proba(x_test_count)
print(log_loss(y_test, prob))

1.0
9.190668028604924e-15


In [ ]:
u

array(['軍事', '經濟', '教育', '體育', '醫藥', '計算機', '環境', '交通', '藝術', '政治'],
      dtype=object)

In [ ]:
p = input("write:")
test = vec.transform([newscut(p)])
prob = clf.predict_proba(test)[0]
for i in range(len(u)):
    print(u[i], "的機率:", round(prob[i], 3)) 

KeyboardInterrupt: ignored

In [ ]:
!pip install opencc-python-reimplemented
#安裝簡體轉繁體的軟體

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2twp')  # convert from Simplified Chinese to Traditional Chinese
# can also set conversion by calling set_conversion
# cc.set_conversion('s2tw')
to_convert = '开放中文转换'
converted = cc.convert(to_convert)
converted